In [1]:
import copy
import re
import pandas as pd
import numpy as np
from experiments import ExperimentModule
from experiments import (
    save_plots, 
    write_row, 
    write_opcounts,
    load_circuit,
    create_dir,
    get_random_state
)
from sanchez_ansatz import SanchezAnsatz
from qiskit import transpile
import matplotlib.pyplot as plt
from itertools import product
from datetime import datetime
from typing import Tuple

%matplotlib inline

In [2]:
results_dir = "results/runs_shell_2"
run_range = list(range(1, 11))
#qubit_range = [3, 4, 5, 6, 7, 8]
qubit_range = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# EPS values are mapped separately
#eps_values = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
eps_values = [0.5, 0.1, 0.05, 0.01]

In [3]:
def load_results(results_dir, state_type, qubit_range, eps_values) -> Tuple[dict,dict]:
    #[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
    original_results_dict = {
        "num_qubits": [],
        0.5: [],
        0.1: [],
        0.05: [],
        0.01: [],
    }

    modified_results_dict = copy.deepcopy(original_results_dict)

    for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
        if not num_qubits in original_results_dict["num_qubits"]: 
            original_results_dict["num_qubits"] += [num_qubits]
            modified_results_dict["num_qubits"] += [num_qubits]

        original_results_dict[eps] += [[]]
        modified_results_dict[eps] += [[]]

        for run_idx in run_range:
            fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
            data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
            original_results_dict[eps][-1] += [data_original.round(4)]

            fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
            data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
            modified_results_dict[eps][-1]+= [data_modified.round(4)]
    
    return original_results_dict, modified_results_dict


def compute_statistics(results_dict: dict) -> Tuple[dict, dict]: 
    default_dict = {
        "num_qubits": qubit_range,
        0.5: [],
        0.1: [],
        0.05: [],
        0.01: [],
    }
    mean_results_dict = copy.deepcopy(default_dict)
    stds_results_dict = copy.deepcopy(default_dict)

    #stds_original_results_dict = copy.deepcopy(default_dict)
    #stds_modified_results_dict = copy.deepcopy(default_dict)

    for eps in eps_values:
        mean_results_dict[eps] = np.mean(results_dict[eps], axis=1)
        stds_results_dict[eps] = np.std(results_dict[eps], axis=1).round(2)

        #mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
        #stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)
    return mean_results_dict, stds_results_dict


def to_dataframe(results_dict: dict) -> pd.DataFrame:
    df_results = pd.DataFrame(results_dict)
    df_results = df_results.set_index("num_qubits")
    return df_results

### Normal distribution $(\mu = 0.5, \sigma = 0.4)$

In [5]:
state_type = "normal"

In [9]:
original_results_dict, modified_results_dict = load_results(results_dir, state_type, qubit_range, eps_values)

mean_original_results_dict, stds_original_results_dict = compute_statistics(original_results_dict)
mean_modified_results_dict, stds_modified_results_dict = compute_statistics(modified_results_dict)

df_mean_original = to_dataframe(mean_original_results_dict)
df_stds_original = to_dataframe(stds_modified_results_dict)

df_mean_modified = to_dataframe(mean_modified_results_dict)
df_stds_modified = to_dataframe(stds_modified_results_dict)

In [10]:
display(df_mean_original)
display(df_mean_modified)

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.02650,0.02650,0.88666,0.95205
4,0.02770,0.02770,0.00560,0.94229
5,0.02670,0.02670,0.00650,0.00600
6,0.02590,0.02590,0.00650,0.01304
7,0.02550,0.02550,0.00640,0.00304
8,0.02520,0.02520,0.10578,0.00240
9,0.02510,0.31747,0.00640,0.00249
10,0.12258,0.02510,0.10570,0.00275
11,0.02500,0.02500,0.00633,0.00333


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.02650,0.02650,0.81050,0.82833
4,0.02750,0.02750,0.00562,0.97106
5,0.02650,0.02650,0.75030,0.00303
6,0.11810,0.02580,0.00650,0.75010
7,0.02530,0.02530,0.50320,0.00304
8,0.02510,0.02510,0.00640,0.00201
9,0.02500,0.02500,0.00640,0.10385
10,0.06900,0.02490,0.00630,0.75047
11,0.02490,0.12238,0.77536,0.10156


In [11]:
df_mean_original < df_mean_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,False,False,False,False
4,False,False,True,True
5,False,False,True,False
6,True,False,False,True
7,False,False,True,False
8,False,False,False,False
9,False,False,False,True
10,False,False,False,True
11,False,True,True,True


### $log$-Normal Distribution $(s= 1, \mu = 0.1, \sigma = 0.3)$

### Bimodal Distribution $(\mu_1 = 0.25, \sigma_1 = 0.3; \mu_2 = 0.75, \sigma_2 = 0.6)$

### Triangular $(c=1, \mu = 0.5, \sigma = 0.5)$

### Laplace $(\mu = 0.5, \sigma = 0.2)$